The goal here is to automatically create graphs with filter functions projecting onto single variables then pairs of variables for Summer 2001-2013

In [1]:
import kmapper as km
from kmapper.plotlyviz import *

In [2]:
import sklearn
#from sklearn import ensemble
# DBSCAN from sklearn for clustering algorithms
from sklearn.cluster import DBSCAN
# PCA from sklearn for projection/lens creation
from sklearn.decomposition import PCA
# from sklearn.manifold import MDS

# scipy for interpolation
# import scipy 
# from scipy.interpolate import *

In [3]:
# Plotly and Dash
import plotly.graph_objs as go
from kmapper.plotlyviz import plotlyviz
from kmapper.plotlyviz import *
from ipywidgets import (HBox, VBox)
import dash_html_components as html
import dash_core_components as dcc
import dash
from ipywidgets import interactive, HBox, VBox, widgets, interact
import warnings
warnings.filterwarnings("ignore")

Set file path for data to read in here

In [4]:
import pandas as pd
from itertools import combinations
predicted_df = pd.read_csv(r"C:\Users\killiad\Documents\Senior\REU\Data\season_and_year.csv")

In [5]:
### Creating three main time spans and two overlapping time spans,
### a total of five time spans

# defining different time periods
# first decade
time_dec1 = [1993, 1994, 1995, 1997, 1998, 1999, 2000]
# second decade
time_dec2 = [2001, 2002,2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013]
# third decade
time_dec3 = [2014, 2015, 2016, 2017, 2018, 2019, 2020]
# overlap time periods for continuity
time_overlap1 = [1998, 1999, 2000, 2001, 2002,2003, 2004]
time_overlap2 = [2010, 2011, 2012, 2013, 2014, 2015, 2016]
time_list = [time_dec2]

time_list_names = ['01-13']
stratum_list = [1, 2, 3, 4, 5, 6, 7, 9]
#Season_names = ["SPRING", "SUMMER", "FALL", "WINTER"]
Season_names = ["SUMMER"]

nonempty_stratum = []
for i in stratum_list:
    if predicted_df[predicted_df['STRATUM'] == i].shape[0]!=0:
        nonempty_stratum.append(i)
print(nonempty_stratum) 


df_stratum_season_time_dict_list = []
df_stratum_season_time_dict = {}
s=""
for i in range(len(time_list)):
    for j in nonempty_stratum:
        for k in Season_names:
            s=str("Stratum ") + str(j)+ " " + k + " " + time_list_names[i] + str(": ")
            df_stratum_season_time_dict[s] = predicted_df[(predicted_df['YEAR'].isin(time_list[i])) &
                                                             (predicted_df['STRATUM'].isin([j])) & 
                                                             (predicted_df['SEASON'].isin([k]))]
            df_stratum_season_time_dict_list.append(df_stratum_season_time_dict)
            df_stratum_season_time_dict = {}
            s=""
print(len(df_stratum_season_time_dict_list))

[1, 2, 3]
3


In [30]:
def cluster_fun(dict_df, DBSCAN_EPSILON = 20, DBSCAN_MIN_SAMPLES = 1, N_CUBES = [7,7], PERC_OVERLAP = [.5,.5]):
    """

    """
    
    
    keys = list(dict_df.keys())
    print(keys)
    X = dict_df.get(keys[0])
    X = X[["PredictedWDP", "PredictedSECCHI", "PredictedTEMP", "PredictedDO", 
           "PredictedTURB", "PredictedVEL", "PredictedTP", 
           "PredictedTN", "PredictedSS", "PredictedCHLcal"]]
    if X.shape[0]<DBSCAN_MIN_SAMPLES:
        #print(X)
        print("Not enough data to cluster in ", keys, "_size = ", X.shape[0])
        print("DBSCAN_MIN_SAMPLES", DBSCAN_MIN_SAMPLES)
        return([DBSCAN_MIN_SAMPLES, X.shape[0]])
    
    
    db = DBSCAN(eps=20, min_samples=2).fit(X)
    # Number of clusters in labels, ignoring noise if present.
    labels = db.labels_
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)

    print('Estimated number of clusters: %d' % n_clusters_)
    print('Estimated number of noise points: %d' % n_noise_)
    
    return(db)

    
    
def mapper_pca_func(dict_df, projection_variable, DBSCAN_EPSILON = 20, DBSCAN_MIN_SAMPLES = 1, 
                    N_CUBES = [7,7], PERC_OVERLAP = [.5,.5]):
    """
    str(df_stratum_season_time_dict_list[1].keys())
    """
    
    # 
    keys = list(dict_df.keys())
    print(keys)
    X = dict_df.get(keys[0])
    X = X[["PredictedWDP", "PredictedSECCHI", "PredictedTEMP", "PredictedDO", 
           "PredictedTURB", "PredictedVEL", "PredictedTP", 
           "PredictedTN", "PredictedSS", "PredictedCHLcal"]]
    
    continuous_variables = ["PredictedWDP", "PredictedSECCHI", "PredictedTEMP", "PredictedDO", 
           "PredictedTURB", "PredictedVEL", "PredictedTP", 
           "PredictedTN", "PredictedSS", "PredictedCHLcal"]
    var_to_index = {continuous_variables[i] : i for i in range(len(continuous_variables))}
    
    projected_vars = projection_variable
    projected_var_indices = [var_to_index[var] for var in projected_vars]
    filter_var_indices = [var_to_index[var] for var in projection_variable]
    
    if X.shape[0]<10:
        #print(X)
        print("Not enough data in ", keys, "_size = ", X.shape[0])
        return(X.shape[0])
    
    X.reset_index(drop = True, inplace = True)

    # defining clustering and kmapper parameters
    DBSCAN_EPSILON = DBSCAN_EPSILON
    DBSCAN_MIN_SAMPLES = DBSCAN_MIN_SAMPLES
    N_CUBES = N_CUBES
    PERC_OVERLAP = PERC_OVERLAP
    
    # create instance of clustering alg
    cluster_alg = sklearn.cluster.DBSCAN(eps=DBSCAN_EPSILON, min_samples=DBSCAN_MIN_SAMPLES, metric='euclidean')

    # Instantiate kepler mapper object
    mapper = km.KeplerMapper(verbose=0)
    
    # defining filter function as a projection
    lens = mapper.project(np.array(X), projection = filter_var_indices, scaler = None)

    # Generate the simplicial complex
    scomplex = mapper.map(lens, X, cover=km.Cover(n_cubes=N_CUBES, perc_overlap=PERC_OVERLAP), clusterer=cluster_alg,
                         remove_duplicate_nodes=True)  
    summary_variable = mapper.project(np.array(X), projection=projected_var_indices, scaler=None)

    pl_brewer = [[0.0, '#006837'],
             [0.1, '#1a9850'],
             [0.2, '#66bd63'],
             [0.3, '#a6d96a'],
             [0.4, '#d9ef8b'],
             [0.5, '#ffffbf'],
             [0.6, '#fee08b'],
             [0.7, '#fdae61'],
             [0.8, '#f46d43'],
             [0.9, '#d73027'],
             [1.0, '#a50026']]

    color_values = lens[:,0] - lens[:,0].min()
    color_function_name = projection_variable[0]
    my_colorscale = pl_brewer
    kmgraph,  mapper_summary, colorf_distribution = get_mapper_graph(scomplex, 
                                                                     color_values=color_values,  
                                                                     color_function_name=color_function_name, 
                                                                     colorscale=my_colorscale)

    bgcolor = 'rgba(10,10,10, 0.9)'
    # y_gridcolor = 'rgb(150,150,150)'# on a black background the gridlines are set on  grey

    plotly_graph_data = plotly_graph(kmgraph, graph_layout='fr', colorscale=my_colorscale, 
                                     factor_size=2.5, edge_linewidth=0.5)
    plot_title = str(keys[0]) + str(DBSCAN_EPSILON) + str(DBSCAN_EPSILON) + ', MIN_SAMPLES ' + str(DBSCAN_MIN_SAMPLES) 
    # plot_title = 'Pool 13, Summer 1993-1999; Epsilon ' + str(DBSCAN_EPSILON) + ', MIN_SAMPLES ' + str(DBSCAN_MIN_SAMPLES) 
    layout = plot_layout(title=plot_title,  
                         width=620, height=570,
                         annotation_text=get_kmgraph_meta(mapper_summary),  
                         bgcolor=bgcolor)

    # FigureWidget is responsible for event listeners

    fw_graph = go.FigureWidget(data=plotly_graph_data, layout=layout)
    fw_hist = node_hist_fig(colorf_distribution, bgcolor=bgcolor)
    fw_summary = summary_fig(mapper_summary, height=300)

    dashboard = hovering_widgets(kmgraph, 
                                 fw_graph, 
                                 bgcolor=bgcolor, 
                                 member_textbox_width=600)

    # DESIRED FILE PATH, CHANGE TO FIT YOUR LOCAL MACHINE
    directory_path = "Users\\killiad\\Documents\\Senior\\REU\\Mapper\\Projection_2\\"
    
    #Update the fw_graph colorbar, setting its title:
    fw_graph.data[1].marker.colorbar.title = 'dist to<br>x-min'
    #html_output_path = directory_path + str(keys[0]) + 'PCA_2' + 'all_var_' + 'Eps_' + str(DBSCAN_EPSILON) +'MinS_' + str(DBSCAN_MIN_SAMPLES) + 'NCUBES_' + str(N_CUBES[0]) + 'PEROvLp_' + str(PERC_OVERLAP[0]) + '.html'
    html_output_path = directory_path + str(keys[0]) + 'Projection_2_' + str(var) + '_Eps_' + str(DBSCAN_EPSILON) +'MinS_' + str(DBSCAN_MIN_SAMPLES) + 'NCUBES_' + str(N_CUBES) + 'PEROvLp_' + str(PERC_OVERLAP) + '.html'
    html_output_path = html_output_path.replace(" ", "-")
    html_output_path = html_output_path.replace(":", "_")
    html_output_path = "C:\\" + html_output_path
    mapper.visualize(scomplex, color_values=color_values, color_function_name=color_function_name, path_html=html_output_path,
                    lens = summary_variable, lens_names = projected_vars)
    return scomplex, X

In [25]:
dbclus_dict = {}
for i in df_stratum_season_time_dict_list:
    dbclus_dict[str(list(i.keys()))]=cluster_fun(i)

['Stratum 1 SUMMER 01-13: ']
Estimated number of clusters: 2
Estimated number of noise points: 8
['Stratum 2 SUMMER 01-13: ']
Estimated number of clusters: 4
Estimated number of noise points: 10
['Stratum 3 SUMMER 01-13: ']
Estimated number of clusters: 20
Estimated number of noise points: 97


Single variable projection

In [29]:
continuous_variables = ["PredictedWDP", "PredictedSECCHI", "PredictedTEMP", "PredictedDO", 
           "PredictedTURB", "PredictedVEL", "PredictedTP", 
           "PredictedTN", "PredictedSS", "PredictedCHLcal"]
n_cubes = [15]
perc_overlap = [.35]
mapper_pca_output_dict = {}
mapper_pca_output_df = {}
for cubes in n_cubes:
    for perc in perc_overlap:
        for i in df_stratum_season_time_dict_list:
            for var in continuous_variables:
                print(var)
                scomplex, X = mapper_pca_func(i, [var], N_CUBES = cubes, PERC_OVERLAP = perc)
                mapper_pca_output_dict[str(list(i.keys()))] = scomplex
                mapper_pca_output_df[str(list(i.keys()))] = X

PredictedWDP
['Stratum 1 SUMMER 01-13: ']
PredictedSECCHI
['Stratum 1 SUMMER 01-13: ']
PredictedTEMP
['Stratum 1 SUMMER 01-13: ']
PredictedDO
['Stratum 1 SUMMER 01-13: ']
PredictedTURB
['Stratum 1 SUMMER 01-13: ']
PredictedVEL
['Stratum 1 SUMMER 01-13: ']
PredictedTP
['Stratum 1 SUMMER 01-13: ']
PredictedTN
['Stratum 1 SUMMER 01-13: ']
PredictedSS
['Stratum 1 SUMMER 01-13: ']
PredictedCHLcal
['Stratum 1 SUMMER 01-13: ']
PredictedWDP
['Stratum 2 SUMMER 01-13: ']
PredictedSECCHI
['Stratum 2 SUMMER 01-13: ']
PredictedTEMP
['Stratum 2 SUMMER 01-13: ']
PredictedDO
['Stratum 2 SUMMER 01-13: ']
PredictedTURB
['Stratum 2 SUMMER 01-13: ']
PredictedVEL
['Stratum 2 SUMMER 01-13: ']
PredictedTP
['Stratum 2 SUMMER 01-13: ']
PredictedTN
['Stratum 2 SUMMER 01-13: ']
PredictedSS
['Stratum 2 SUMMER 01-13: ']
PredictedCHLcal
['Stratum 2 SUMMER 01-13: ']
PredictedWDP
['Stratum 3 SUMMER 01-13: ']
PredictedSECCHI
['Stratum 3 SUMMER 01-13: ']
PredictedTEMP
['Stratum 3 SUMMER 01-13: ']
PredictedDO
['Stratum 

In [32]:
continuous_variables = ["PredictedWDP", "PredictedSECCHI", "PredictedTEMP", "PredictedDO", 
           "PredictedTURB", "PredictedVEL", "PredictedTP", 
           "PredictedTN", "PredictedSS", "PredictedCHLcal"]
n_cubes = [5]
perc_overlap = [.35]
mapper_pca_output_dict = {}
mapper_pca_output_df = {}
for cubes in n_cubes:
    for perc in perc_overlap:
        for i in df_stratum_season_time_dict_list:
            for var in combinations(continuous_variables, 2):
                print(var)
                scomplex, X = mapper_pca_func(i, np.array(var), N_CUBES = [cubes,cubes], PERC_OVERLAP = [perc,perc])
                mapper_pca_output_dict[str(list(i.keys()))] = scomplex
                mapper_pca_output_df[str(list(i.keys()))] = X

('PredictedWDP', 'PredictedSECCHI')
['Stratum 1 SUMMER 01-13: ']
('PredictedWDP', 'PredictedTEMP')
['Stratum 1 SUMMER 01-13: ']
('PredictedWDP', 'PredictedDO')
['Stratum 1 SUMMER 01-13: ']
('PredictedWDP', 'PredictedTURB')
['Stratum 1 SUMMER 01-13: ']
('PredictedWDP', 'PredictedVEL')
['Stratum 1 SUMMER 01-13: ']
('PredictedWDP', 'PredictedTP')
['Stratum 1 SUMMER 01-13: ']
('PredictedWDP', 'PredictedTN')
['Stratum 1 SUMMER 01-13: ']
('PredictedWDP', 'PredictedSS')
['Stratum 1 SUMMER 01-13: ']
('PredictedWDP', 'PredictedCHLcal')
['Stratum 1 SUMMER 01-13: ']
('PredictedSECCHI', 'PredictedTEMP')
['Stratum 1 SUMMER 01-13: ']
('PredictedSECCHI', 'PredictedDO')
['Stratum 1 SUMMER 01-13: ']
('PredictedSECCHI', 'PredictedTURB')
['Stratum 1 SUMMER 01-13: ']
('PredictedSECCHI', 'PredictedVEL')
['Stratum 1 SUMMER 01-13: ']
('PredictedSECCHI', 'PredictedTP')
['Stratum 1 SUMMER 01-13: ']
('PredictedSECCHI', 'PredictedTN')
['Stratum 1 SUMMER 01-13: ']
('PredictedSECCHI', 'PredictedSS')
['Stratum 1 SU

('PredictedTP', 'PredictedCHLcal')
['Stratum 3 SUMMER 01-13: ']
('PredictedTN', 'PredictedSS')
['Stratum 3 SUMMER 01-13: ']
('PredictedTN', 'PredictedCHLcal')
['Stratum 3 SUMMER 01-13: ']
('PredictedSS', 'PredictedCHLcal')
['Stratum 3 SUMMER 01-13: ']


In [ ]:
import json


json = json.dumps(mapper_pca_output_dict)
f = open(r"C:\Users\killiad\Documents\Senior\REU\Mapper\PCA_1_NO_COND\TDA1PCA15Cubes35perc.json","w")
f.write(json)
f.close()

In [ ]:
continuous_variables = ["PredictedWDP", "PredictedSECCHI", "PredictedTEMP", "PredictedDO", 
           "PredictedTURB", "PredictedVEL", "PredictedTP", 
           "PredictedTN", "PredictedSS", "PredictedCHLcal"]
list(combinations(continuous_variables, 2))

In [ ]:
import pickle as pk
pickle = pk.dump(mapper_pca_output_df, open(r"C:\Users\killiad\Documents\Senior\REU\Mapper\PCA_1_NO_COND\TDA1PCA15Cubes35perc.p", "wb"))